In [1]:
import matplotlib.pyplot as plt
import numpy as np
from custom_sepsis import *

/Users/luisastue/miniconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
import numpy as np
import pandas as pd
from itertools import product
from multiprocessing import Pool, cpu_count

# Define parameter grids for each optimization goal
param_grids = {
    "sample_efficiency": {
        "buffer_size": [50000, 100000],
        "batch_size": [16, 32],
        "train_freq": [1, 2],
        "gradient_steps": [1, 5],
        "exploration_fraction": [0.2, 0.3],
        "exploration_final_eps": [0.01, 0.05],
        "target_update_interval": [100, 200],
    },
    "asymptotic_rewards": {
        "buffer_size": [500000, 1000000],
        "batch_size": [128, 256],
        "train_freq": [4, 8],
        "gradient_steps": [10, 20],
        "exploration_fraction": [0.05, 0.1],
        "exploration_final_eps": [0.01, 0.02],
        "target_update_interval": [500, 1000],
    },
    "cumulative_rewards": {
        "buffer_size": [100000, 500000],
        "batch_size": [64, 128],
        "train_freq": [2, 4],
        "gradient_steps": [5, 10],
        "exploration_fraction": [0.1, 0.2],
        "exploration_final_eps": [0.01, 0.05],
        "target_update_interval": [200, 500],
    },
}
results = []


In [ ]:
tasks = create_tasks(param_grids)

# Use a single pool to process all tasks
num_cpus = cpu_count()
with Pool(num_cpus) as pool:
    results = pool.map(process_task, tasks)
    print("results ", len(results))

# Process results as needed
print("Total experiments completed:", len(results))

/Users/luisastue/miniconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/luisastue/miniconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/luisastue/miniconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/luisastue/miniconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/luisastue/miniconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: Us

In [3]:
len(results)

0

In [ ]:

# Analyze results
results_df = pd.DataFrame([
    {
        "params": res[0],
        "sample_efficiency": np.argmax(np.array(res[1]) >= -0.5) if np.any(np.array(res[1]) >= -0.5) else float('inf'),
        "asymptotic_rewards": np.mean(res[1][-100:]),
        "cumulative_rewards": np.sum(res[1]),
    }
    for res in results
])

# Normalize scores and rank
results_df["sample_efficiency_score"] = 1 / results_df["sample_efficiency"]  # Lower is better
results_df["asymptotic_rewards_score"] = results_df["asymptotic_rewards"]
results_df["cumulative_rewards_score"] = results_df["cumulative_rewards"]

# Extract best parameters for each goal
best_sample_efficiency = results_df.iloc[results_df["sample_efficiency_score"].idxmax()]["params"]
best_asymptotic_rewards = results_df.iloc[results_df["asymptotic_rewards_score"].idxmax()]["params"]
best_cumulative_rewards = results_df.iloc[results_df["cumulative_rewards_score"].idxmax()]["params"]

# Print results
print("Best parameters for sample efficiency:", best_sample_efficiency)
print("Best parameters for asymptotic rewards:", best_asymptotic_rewards)
print("Best parameters for cumulative rewards:", best_cumulative_rewards)


In [ ]:
import matplotlib.pyplot as plt

# Function to calculate moving average
def moving_average(data, window_size=50):
    return np.convolve(data, np.ones(window_size) / window_size, mode='valid')

# Re-train the best models to gather their episode rewards
def train_best_model(params):
    true_env = ...  # Replace with your environment initialization
    callback = CustomLoggingCallback()

    model = DQN(
        "MlpPolicy",
        true_env,
        buffer_size=params["buffer_size"],
        batch_size=params["batch_size"],
        train_freq=params["train_freq"],
        gradient_steps=params["gradient_steps"],
        exploration_fraction=params["exploration_fraction"],
        exploration_final_eps=params["exploration_final_eps"],
        target_update_interval=params["target_update_interval"],
        learning_rate=1e-4,
        gamma=0.99,
        verbose=0,
    )

    model.learn(total_timesteps=200000, callback=callback)
    episode_rewards = [np.sum(episode.rewards) for episode in callback.episodes]
    return episode_rewards

# Train and collect rewards for the three best models
best_models = {
    "Sample Efficiency": best_sample_efficiency,
    "Asymptotic Rewards": best_asymptotic_rewards,
    "Cumulative Rewards": best_cumulative_rewards,
}

rewards_data = {}

for model_name, params in best_models.items():
    rewards_data[model_name] = train_best_model(params)

# Plot moving averages of the rewards
plt.figure(figsize=(12, 6))
for model_name, rewards in rewards_data.items():
    moving_avg = moving_average(rewards, window_size=50)
    plt.plot(moving_avg, label=f"{model_name} (Moving Average)")

plt.title("Moving Average of Rewards for Best Models")
plt.xlabel("Episode")
plt.ylabel("Reward (Moving Average)")
plt.legend()
plt.grid()
plt.show()
